In [2]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/heather/code/loicmorel/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [3]:
import pandas as pd
import numpy as np
from app.data_collection.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [4]:
file = 'data/ohlcv_SOLUSDT_1h_from_1597125600000_to_1651507200000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2020-08-11 06:00:00,2.8500,3.4700,2.8500,2.9515,20032.26,2020-08-11 06:59:59.999000064,18485.250000
1,2020-08-11 07:00:00,2.9515,3.1355,2.8800,2.9224,42069.37,2020-08-11 07:59:59.999000064,18485.291667
2,2020-08-11 08:00:00,2.9626,3.0000,2.9144,2.9600,24280.76,2020-08-11 08:59:59.999000064,18485.333333
3,2020-08-11 09:00:00,2.9600,2.9736,2.8500,2.8543,26371.23,2020-08-11 09:59:59.999000064,18485.375000
4,2020-08-11 10:00:00,2.8566,2.9329,2.8433,2.8976,26685.94,2020-08-11 10:59:59.999000064,18485.416667
...,...,...,...,...,...,...,...,...
15084,2022-05-02 12:00:00,86.7700,87.6800,86.5800,87.0500,67880.56,2022-05-02 12:59:59.999000064,19114.500000
15085,2022-05-02 13:00:00,87.0500,88.5100,86.2500,87.8600,219874.72,2022-05-02 13:59:59.999000064,19114.541667
15086,2022-05-02 14:00:00,87.8500,88.9400,86.9500,88.2600,146423.33,2022-05-02 14:59:59.999000064,19114.583333
15087,2022-05-02 15:00:00,88.2700,88.4900,87.1000,87.7200,101210.49,2022-05-02 15:59:59.999000064,19114.625000


In [5]:
from app.indicators import find_swings
data_ = find_swings(data)

In [6]:
from app.indicators import classify_swings
data__ = classify_swings(data_)

In [7]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,3.4700,2.85,3.4700,0.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
1,3.1355,2.88,3.1355,NaN,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
2,3.4700,0.00,3.4700,-1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
3,3.4700,0.00,3.4700,-1.0,1,False,True,0.00,3.47,0.00,3.47,False,False,True,False
4,3.4700,0.00,3.4700,-1.0,2,False,True,0.00,3.47,0.00,0.00,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15084,90.7700,88.67,90.7700,-1.0,5,False,True,0.00,90.77,0.00,0.00,False,False,False,False
15085,90.7700,86.29,86.2900,1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
15086,90.7700,86.29,86.2900,1.0,1,True,False,86.29,0.00,86.29,0.00,True,False,False,False
15087,90.7700,86.29,86.2900,1.0,2,True,False,86.29,0.00,0.00,0.00,False,False,False,False


In [8]:
data__=data__*1

In [9]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,3.4700,2.85,3.4700,0.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
1,3.1355,2.88,3.1355,NaN,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
2,3.4700,0.00,3.4700,-1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
3,3.4700,0.00,3.4700,-1.0,1,0,1,0.00,3.47,0.00,3.47,0,0,1,0
4,3.4700,0.00,3.4700,-1.0,2,0,1,0.00,3.47,0.00,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15084,90.7700,88.67,90.7700,-1.0,5,0,1,0.00,90.77,0.00,0.00,0,0,0,0
15085,90.7700,86.29,86.2900,1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
15086,90.7700,86.29,86.2900,1.0,1,1,0,86.29,0.00,86.29,0.00,1,0,0,0
15087,90.7700,86.29,86.2900,1.0,2,1,0,86.29,0.00,0.00,0.00,0,0,0,0


In [10]:
df=pd.merge(data,data__,left_index=True,right_index=True)

In [11]:
df=df[['o_ts','Open','High','Low','Close','Volume','LL','HL','HH','LH']]

In [12]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH
0,2020-08-11 06:00:00,2.8500,3.4700,2.8500,2.9515,20032.26,0,0,0,0
1,2020-08-11 07:00:00,2.9515,3.1355,2.8800,2.9224,42069.37,0,0,0,0
2,2020-08-11 08:00:00,2.9626,3.0000,2.9144,2.9600,24280.76,0,0,0,0
3,2020-08-11 09:00:00,2.9600,2.9736,2.8500,2.8543,26371.23,0,0,1,0
4,2020-08-11 10:00:00,2.8566,2.9329,2.8433,2.8976,26685.94,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
15084,2022-05-02 12:00:00,86.7700,87.6800,86.5800,87.0500,67880.56,0,0,0,0
15085,2022-05-02 13:00:00,87.0500,88.5100,86.2500,87.8600,219874.72,0,0,0,0
15086,2022-05-02 14:00:00,87.8500,88.9400,86.9500,88.2600,146423.33,1,0,0,0
15087,2022-05-02 15:00:00,88.2700,88.4900,87.1000,87.7200,101210.49,0,0,0,0


In [13]:
df['sum'] = df[['HH','HL','LL','LH']].sum(axis=1)
df['NP']=df['sum'].apply(lambda x:1 if x==0 else 0)
df.drop(columns='sum',inplace=True)

In [14]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH,NP
0,2020-08-11 06:00:00,2.8500,3.4700,2.8500,2.9515,20032.26,0,0,0,0,1
1,2020-08-11 07:00:00,2.9515,3.1355,2.8800,2.9224,42069.37,0,0,0,0,1
2,2020-08-11 08:00:00,2.9626,3.0000,2.9144,2.9600,24280.76,0,0,0,0,1
3,2020-08-11 09:00:00,2.9600,2.9736,2.8500,2.8543,26371.23,0,0,1,0,0
4,2020-08-11 10:00:00,2.8566,2.9329,2.8433,2.8976,26685.94,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
15084,2022-05-02 12:00:00,86.7700,87.6800,86.5800,87.0500,67880.56,0,0,0,0,1
15085,2022-05-02 13:00:00,87.0500,88.5100,86.2500,87.8600,219874.72,0,0,0,0,1
15086,2022-05-02 14:00:00,87.8500,88.9400,86.9500,88.2600,146423.33,1,0,0,0,0
15087,2022-05-02 15:00:00,88.2700,88.4900,87.1000,87.7200,101210.49,0,0,0,0,1


In [15]:
df.set_index('o_ts',inplace=True)

In [16]:
df.to_csv('data/transformed_data/SOLUSDT_1h_transformed.csv')

In [17]:
df[['HH','HL','LL','LH','NP']].value_counts()

HH  HL  LL  LH  NP
0   0   0   0   1     11668
            1   0       991
    1   0   0   0       984
    0   1   0   0       723
1   0   0   0   0       723
dtype: int64

In [18]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    return df_sample

In [19]:
assert subsample_sequence(df, 10).shape == (10, 10)
assert subsample_sequence(df, 10000).shape == (10000, 10)

In [20]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1][['LL','HL','HH','LH','NP']]
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

In [21]:
(X_sample, y_sample) = split_subsample_sequence(df, 10)
assert X_sample.shape == (9,10)
assert y_sample.shape == (5,)

In [22]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [23]:
X, y = get_X_y(df, 1000, 21)
print(X.shape)
print(y.shape)

(1000, 20, 10)
(1000, 5)


In [24]:
assert np.isnan(X).sum() == 0

In [25]:
len_ = int(0.8*df.shape[0])
data_train = df[:len_]
data_test = df[len_:]

In [26]:
X_train, y_train = get_X_y(data_train, 1000, 21)
X_test, y_test = get_X_y(data_test, 100, 21)

In [27]:
assert X_train.shape == (1000, 20, 10)

In [28]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [29]:


def init_model():
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=True))
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=False))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])
    
    return model

In [30]:
model = init_model()
model.summary()

2022-05-06 13:02:02.384347: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 13:02:02.410918: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, None, 10)          21        
_________________________________________________________________
gru (GRU)                    (None, None, 20)          1920      
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                2520      
_________________________________________________________________
dense (Dense)                (None, 50)                1050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 5,766
Trainable params: 5,745
Non-trainable params: 21
_____________________________________________________

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='accuracy', patience=10)

In [32]:
class_weight = {0: 5.,
                1: 5.,
                2: 5.,
                3: 5.,
                4: 1.}

In [33]:
model.fit(X_train, y_train, 
          epochs=75, 
          batch_size=64, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,class_weight=class_weight)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/75
13/13 [==============================] - 2s 42ms/step - loss: 3.1126 - accuracy: 0.6488 - val_loss: 1.2421 - val_accuracy: 0.5750
Epoch 2/75
13/13 [==============================] - 0s 12ms/step - loss: 2.7974 - accuracy: 0.5300 - val_loss: 1.0120 - val_accuracy: 0.5800
Epoch 3/75
13/13 [==============================] - 0s 12ms/step - loss: 2.5808 - accuracy: 0.5200 - val_loss: 1.0776 - val_accuracy: 0.3700
Epoch 4/75
13/13 [==============================] - 0s 12ms/step - loss: 2.5029 - accuracy: 0.4837 - val_loss: 1.0554 - val_accuracy: 0.3650
Epoch 5/75
13/13 [==============================] - 0s 11ms/step - loss: 2.4271 - accuracy: 0.4187 - val_loss: 0.9306 - val_accuracy: 0.4750
Epoch 6/75
13/13 [==============================] - 0s 11ms/step - loss: 2.3664 - accuracy: 0.4588 - val_loss: 1.0101 - val_accuracy: 0.3800
Epoch 7/75
13/13

In [34]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 0.9322 - accuracy: 0.5100


[0.932233452796936, 0.5099999904632568]

In [35]:
np.round(model.predict_proba(X_test),2)

/Users/heather/.pyenv/versions/3.8.12/envs/crypto_assistant/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.01, 0.44, 0.01, 0.01, 0.54],
       [0.53, 0.13, 0.01, 0.15, 0.18],
       [0.  , 0.02, 0.23, 0.33, 0.41],
       [0.27, 0.3 , 0.  , 0.16, 0.27],
       [0.02, 0.46, 0.02, 0.15, 0.36],
       [0.  , 0.  , 0.02, 0.4 , 0.58],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.15, 0.51, 0.07, 0.26],
       [0.  , 0.08, 0.57, 0.09, 0.26],
       [0.  , 0.  , 0.06, 0.23, 0.71],
       [0.01, 0.12, 0.45, 0.15, 0.27],
       [0.75, 0.05, 0.  , 0.12, 0.09],
       [0.56, 0.09, 0.  , 0.19, 0.16],
       [0.02, 0.5 , 0.01, 0.13, 0.33],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.13, 0.56, 0.05, 0.26],
       [0.12, 0.44, 0.  , 0.2 , 0.24],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.08, 0.27, 0.26, 0.39],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.55, 0.09, 0.  ,

In [36]:
model.save('./models/RNN_SOLUSDT_1h.h5')